## Load libraries

In [1]:
import os
import requests
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt # Pendiente instalar

import tensorflow as tf
import tensorflow_data_validation as tfdv
from tensorflow_metadata.proto.v0 import schema_pb2

from tfx.components import CsvExampleGen, ExampleValidator, SchemaGen, StatisticsGen, Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
import pprint
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder

2025-03-02 21:16:12.573920: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-02 21:16:14.333817: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-02 21:16:19.773566: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-02 21:16:22.567127: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-02 21:16:22.652812: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 21:16:28.667338: I tensorflow/core/platform/cpu_feature_guard.cc:

## Load Data

In [2]:
## download the dataset
# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
 #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
 url = 'https://docs.google.com/uc?export= \
 download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
 r = requests.get(url, allow_redirects=True, stream=True)
 open(_data_filepath, 'wb').write(r.content)

# Read the CSV file into a DataFrame
if os.path.exists(_data_filepath):
    df = pd.read_csv(_data_filepath)
    print("Dataframe was loaded")  # Display the first few rows
else:
    print(f"File not found: {_data_filepath}")

Dataframe was loaded


# Configurar el contexto interativo

In [10]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen,ExampleValidator
from tfx.v1.components import ImportSchemaGen

# Especifica la ruta al archivo de base de datos de metadatos SQLite#
metadata_db_uri = 'metadata'

context = InteractiveContext(pipeline_root = metadata_db_uri)

## Generando un ejemplo con los datos

In [13]:
example_gen = CsvExampleGen(input_base="data/covertype/")
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 3
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## Cargando estadísticas del ejemplo

In [14]:
# Dataset statistics
statistics_gen = StatisticsGen(
      examples=example_gen.outputs['examples']
      )
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

## Inferir el esquema

In [15]:
# Infer schema from the statistics output
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,,-
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_3pm',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## Curar el esquema

In [17]:
schema_uri = schema_gen.outputs['schema'].get()[0].uri
schema = tfdv.load_schema_text(f'{schema_uri}/schema.pbtxt')
tfdv.set_domain(schema, "Hillshade_9am", schema_pb2.IntDomain(min=0,max=255))
tfdv.set_domain(schema, "Hillshade_Noon", schema_pb2.IntDomain(min=0,max=255))
tfdv.set_domain(schema, "Slope", schema_pb2.IntDomain(min=0,max=90))

feature = tfdv.get_feature(schema, "Cover_Type",)
feature.type = schema_pb2.BYTES
tfdv.set_domain(schema, "Cover_Type", schema_pb2.StringDomain(value =['0','1','2','3','4','5','6']))

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,,-
'Cover_Type',STRING,required,,'Cover_Type_domain'
'Elevation',INT,required,,-
'Hillshade_3pm',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Cover_Type_domain',"'0', '1', '2', '3', '4', '5', '6'"


## Entornos de Esquema

In [18]:
serv_sampl = df.sample(frac=0.2).drop(['Cover_Type'],axis=1)
serv_sampl.to_csv('data/covertype_serv.csv')
serv_sampl.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type
31711,2846,337,10,90,35,90,199,227,167,1296,Rawah,C7746
43849,3106,216,8,124,4,1387,212,248,172,2466,Commanche,C7757
93442,3409,39,16,750,60,3436,219,203,114,1648,Commanche,C8772
69070,3028,338,13,85,16,4668,192,222,168,553,Rawah,C7746
96712,3119,68,13,85,13,2534,233,214,112,1761,Commanche,C7755


In [20]:
# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Specify that 'tips' feature is not in SERVING environment.
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')

options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)
training_stats = tfdv.generate_statistics_from_csv('data/covertype/covertype_train.csv', stats_options=options)

training_anomalies_with_env = tfdv.validate_statistics(
    training_stats, schema, environment='TRAINING')

tfdv.display_anomalies(training_anomalies_with_env)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [21]:
options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)
serving_stats = tfdv.generate_statistics_from_csv('data/covertype_serv.csv', stats_options=options)

serving_anomalies_with_env = tfdv.validate_statistics(
    serving_stats, schema, environment='SERVING')

tfdv.display_anomalies(serving_anomalies_with_env)

## Actualizar estadisticas con esquema curado

In [22]:

tfdv.write_schema_text(schema, 'mySchema.pbtxt')
schema_gen = ImportSchemaGen(schema_file='mySchema.pbtxt')
context.run(schema_gen)
context.show(schema_gen.outputs['schema'])

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'],
    schema = schema_gen.outputs['schema']
      )
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,,-
'Cover_Type',STRING,required,,'Cover_Type_domain'
'Elevation',INT,required,,-
'Hillshade_3pm',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Cover_Type_domain',"'0', '1', '2', '3', '4', '5', '6'"


In [25]:
example_validator = ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_gen.outputs['schema']
      )
context.run(example_validator)
context.show(schema_gen.outputs['schema'])
context.show(example_validator.outputs['anomalies'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,,-
'Cover_Type',STRING,required,,'Cover_Type_domain'
'Elevation',INT,required,,-
'Hillshade_3pm',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Cover_Type_domain',"'0', '1', '2', '3', '4', '5', '6'"


,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Unexpected data type,Expected data of type: BYTES but got INT


,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Unexpected data type,Expected data of type: BYTES but got INT


In [26]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("processing_for_tfx.py") # Path to preprocessing function file
)

context.run(transform)

running bdist_wheel
running build
running build_py
creating build/lib
copying processing_for_tfx.py -> build/lib
copying get_processed_data.py -> build/lib
installing to /tmp/tmpf1yg85_b
running install
running install_lib
copying build/lib/processing_for_tfx.py -> /tmp/tmpf1yg85_b/.
copying build/lib/get_processed_data.py -> /tmp/tmpf1yg85_b/.
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpf1yg85_b/./tfx_user_code_Transform-0.0+7f04e6d4cbd3cddcc1793179a1ee9587745c53f94798821deedeefea21ecf6c4-

/root/.cache/uv/archive-v0/9RWgtVVkPtQdOKj9wQBgL/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:79: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


creating /tmp/tmpf1yg85_b/tfx_user_code_transform-0.0+7f04e6d4cbd3cddcc1793179a1ee9587745c53f94798821deedeefea21ecf6c4.dist-info/WHEEL
creating '/tmp/tmp1bexper2/tfx_user_code_transform-0.0+7f04e6d4cbd3cddcc1793179a1ee9587745c53f94798821deedeefea21ecf6c4-py3-none-any.whl' and adding '/tmp/tmpf1yg85_b' to it
adding 'get_processed_data.py'
adding 'processing_for_tfx.py'
adding 'tfx_user_code_transform-0.0+7f04e6d4cbd3cddcc1793179a1ee9587745c53f94798821deedeefea21ecf6c4.dist-info/METADATA'
adding 'tfx_user_code_transform-0.0+7f04e6d4cbd3cddcc1793179a1ee9587745c53f94798821deedeefea21ecf6c4.dist-info/WHEEL'
adding 'tfx_user_code_transform-0.0+7f04e6d4cbd3cddcc1793179a1ee9587745c53f94798821deedeefea21ecf6c4.dist-info/top_level.txt'
adding 'tfx_user_code_transform-0.0+7f04e6d4cbd3cddcc1793179a1ee9587745c53f94798821deedeefea21ecf6c4.dist-info/RECORD'
removing /tmp/tmpf1yg85_b
Processing ./metadata/_wheels/tfx_user_code_transform-0.0+7f04e6d4cbd3cddcc1793179a1ee9587745c53f94798821deedeefea21ecf

INFO:tensorflow:Assets written to: metadata/Transform/transform_graph/10/.temp_path/tftransform_tmp/efbce8f0f0264a9f98fc2754b0a42171/assets


INFO:tensorflow:Assets written to: metadata/Transform/transform_graph/10/.temp_path/tftransform_tmp/efbce8f0f0264a9f98fc2754b0a42171/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 10
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [37]:
# Load TFRecord dataset
transformed_example_uri = transform.outputs['transformed_examples'].get()[0].uri
dataset = tfdv.generate_statistics_from_tfrecord(f'{transformed_example_uri}/Split-train/transformed_examples-00000-of-00001.gz')

tfdv.visualize_statistics(dataset)

# 5.1 Metadatos de aprendizaje automático

In [18]:
from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

In [17]:
# Ruta al almacen de metadatos
metadata_connection_config = context.metadata_connection_config

In [19]:
# Creación de una instancia de MetadataStore
store = metadata_store.MetadataStore(metadata_connection_config)

In [20]:
# Recuperar y mostrar todos los tipos de artefactos
artifact_types = store.get_artifact_types()
for artifact_type in artifact_types:
    print(artifact_type.name)

Examples
ExampleStatistics
Schema


In [21]:
# Obtener artefactos de tipo 'Schema'
schema_artifacts = store.get_artifacts_by_type('Schema')

# Mostrar detalles de los artefactos Schema
for artifact in schema_artifacts:
    print(f"Artifact ID: {artifact.id}")
    print(f"URI: {artifact.uri}")

Artifact ID: 3
URI: metadata/SchemaGen/schema/3


In [22]:
if schema_artifacts:
    first_schema_artifact = schema_artifacts[0]
    print(f"Name: {first_schema_artifact.name}")
    print(f"State: {first_schema_artifact.state}")
    print(f"Type: {first_schema_artifact.type_id}")

Name: schema:2025-02-26T02:19:48.957331
State: 2
Type: 18


# 5.2 Seguimiento de artefactos

In [23]:
def get_artifacts_details(store, type_name):
    artifacts = store.get_artifacts_by_type(type_name)
    data = []
    
    for artifact in artifacts:
        data.append({
            'Artifact ID': artifact.id,
            'Type': type_name,
            'URI': artifact.uri
        })

    df = pd.DataFrame(data)
    return df

# 5.3 Obtener artefactos principales

In [25]:
# Se obtienen los artefactos principales para 'Examples'
get_artifacts_details(store, 'Examples')

,Artifact ID,Type,URI
0,1,Examples,metadata/CsvExampleGen/examples/1


In [26]:
# Se obtienen los artefactos principales para 'Schema'
get_artifacts_details(store, 'Schema')

,Artifact ID,Type,URI
0,3,Schema,metadata/SchemaGen/schema/3
